In [2]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
import json
import pandas as pd # this will need to be installed via command line first
import glob
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Method to get all the revelent player attributes for every player currently signed on every NFL team
# The CBS Sports website is used as the source

def GetPlayterAttributes(team: list): # accepts a list ['team-name', 'LOC']
    # Use requests to get the raw HTML response from cbssports.com
    CBSResponse = requests.get(f'https://www.cbssports.com/nfl/teams/{team[1]}/{team[0]}/roster/')
    
    # If we get a normal response proceed with data scraping
    if CBSResponse.status_code == 200:
                 
        # Confirm the request was successful
        print(f"Request succeeded with status code {CBSResponse.status_code}")
        
        # Use Pandas to read the HTML content and put it into a DataFrame
        TableTag = 'TableBase-table'
        DataFrames = pd.read_html(CBSResponse.content, attrs={'class': TableTag})
        
        # Select the relevant data from the DataFrame(offense, defense, and special teams)
        OffenseDF = DataFrames[0][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        DefenseDF = DataFrames[1][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        SpecialDF = DataFrames[2][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        
        # Combine the three DataFrames into one
        FullTeamDF = pd.concat([OffenseDF, DefenseDF, SpecialDF], axis = 0, ignore_index = True)
        
        # Rename the DOB coloumn
        FullTeamDF = FullTeamDF.rename(columns = {'DOB (AGE)' : 'DOB'})
        
        # Create a new column for player age to separate it from DOB
        AgeList = list(FullTeamDF['DOB'].str.slice(-3, -1))
        # Convert the items in the list to integers
        AgeList = [int(s) if s else 22 for s in AgeList] # account for player with no age data
        # Create the new column for player age
        FullTeamDF['AGE'] = AgeList
        # replace the DOB coloumn with just the DOB
        FullTeamDF['DOB'] = FullTeamDF['DOB'].str.split(expand = True)[0]
        
        # Method to clean the player names since CBS puts player news directly in the name cell
        def CleanNameCell(NameText):
            # Split the name cell into a list
            NameText = NameText.split(' ')
            
            # If the name cell doesn't contain player news and no name postfix
            if len(NameText) == 3:
                # Remove the first initial by selecting the rest of the elements
                NameText = NameText[1:]
                # Extract the first name from the first-last name string
                NameText[0] = NameText[0][len(NameText[1]):]
                # Create new string with first and last name
                NameText = ' '.join(NameText)
                
            # The name cell contains a name postfix
            elif len(NameText) == 5:
                # Create new list to store the frist name, last name, postfix
                NameTextPostfix = []
                # Extract the first name from the first-suffix name string, add it to the list
                NameTextPostfix.append(NameText[2][len(NameText[4]):])
                # Add the last name to the list
                NameTextPostfix.append(NameText[1])
                # Add the postfix to the list
                NameTextPostfix.append(NameText[4])
                # Turn the list into a string
                NameText = ' '.join(NameTextPostfix)

            # Otherwise the player name cell contains player news, injury status
            else:
                # Select only the capitilized words from the player cell
                NameTextCapitalized = [s for s in NameText if s and not s[0].islower()]
                # Select the first inital which is always the first item in NameText when there is news, injury status
                First = NameTextCapitalized[0][0]
                # Select the last which is always the second item in NameText when there is news, injury status
                LastName =  NameTextCapitalized[1]
                # Create postfix varaible in case player name has it
                Postfix = ''
                PostfixList = {'Jr.', 'II','III', 'IV'} # set of possible postfixes

                # Iterate through the items in NameTextCapitalzied from the third item to the end
                for s in range(3, (len(NameTextCapitalized) - 1)):
                    # Make sure we're always comparing to the first letter of current item
                    FirstTemp = NameTextCapitalized[s][0]
                    # If the current element first char is the first initial and the next element is last name
                    if NameTextCapitalized[s][0] == FirstTemp and NameTextCapitalized[s + 1].replace(":", "") == LastName:
                        # If we haven't reached the second to last elementand the next next element is a postfix
                        if NameTextCapitalized[s + 2] in PostfixList:
                            # Set the postfix variable to the next next element
                            Postfix = NameTextCapitalized[s + 2].replace(".", "") # remove the period from the postfix

                        # Set the first and last name 
                        First = NameTextCapitalized[s]
                        LastName = NameTextCapitalized[s + 1].replace(":", "") # remove the possible colon from the last name
                        # No break point because the first instance of the full name might not contain a postfix
                        
                        
                # Recreats NameText with the first name, last name
                NameText = [First, LastName]
                # If a postfix was found, add it to the list
                if Postfix:
                    NameText.append(Postfix)
                
                NameText = ' '.join(NameText)

            
            return NameText
        

        # Apply the method to the player names column
        FullTeamDF['Player'] = FullTeamDF['Player'].apply(CleanNameCell)  
        # Sort the DataFrame by player name
        FullTeamDF = FullTeamDF.sort_values(by = ['Player'], ignore_index = True) 
        
        # Add the team location to make it easier to match players up in different files
        FullTeamDF['TeamLOC'] = team[1]
        # Create new column order and apply it
        new_columns = ['Player', 'TeamLOC', 'POS', 'HT', 'WT', 'DOB', 'AGE', 'EXP']
        FullTeamDF = FullTeamDF.loc[:,  new_columns]
        # Rename columns for more consistency
        FullTeamDF.columns = ['Name', 'Team', 'POS', 'HT', 'WT', 'DOB', 'AGE', 'EXP']
        
        # Method to create a unique key for each row
        def CreateKey(player:str, team: str):
            # Split player name into first, last name, possible suffix
            NameList = player.split()
            # Set first and last name
            FirstName, LastName = NameList[0][0], NameList[1]
            # Create the unique key
            key = f'{FirstName}{LastName}{team}'
            
            return key
        
            
        # Create unique key for each row that is represents the player name and team location
        FullTeamDF['Key'] = FullTeamDF.apply(lambda row: CreateKey(row['Name'], row['Team']), axis = 1)
        
        # Create a .csv file with all the player names and their cap hits
        FullTeamDF.to_csv(f'./data/player-attributes-lists/{team[1]}-Player-Attributes-List.csv', index = False, header = False)
        
        """
        # Read the csv back for testing purposes
        FullTeamCSV = pd.read_csv(f'./data/player-attributes-lists/{team[1]}-Player-Attributes-List.csv', header = None)
        FullTeamCSV.columns = ['Name', 'Team', 'POS', 'HT', 'WT', 'DOB', 'AGE', 'EXP', 'Key']
        print(FullTeamCSV)
        """
        return 0

    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {CBSResponse.status_code}")
        return -1



In [4]:
# Method to create a csv of all nfl players and relevent attributes
# Relevent player data is from CBS sports and Wikipedia.

def CombineAttributesLists():
    # The directory containing the player salary data from all 32 nfl teams
    AttributesDataPath = "./data/player-attributes-lists/"
    
    # Get a list of all the files in the directory
    AttributesCSVList = glob.glob(AttributesDataPath + "/*.csv")
    AttributesCSVList = AttributesCSVList[:-1] # do not process the master list
    
    # Create a list to store the individual dataframes
    AttributesDFList = []
    
    # Loop through each file and append it to the list
    for file in AttributesCSVList:
        # Read the file into a dataframe
        df = pd.read_csv(file, header = None, names = ['Name', 'Team', 'POS', 'HT', 'WT', 'DOB', 'AGE', 'EXP', 'Key'])
        # Append the dataframe to the list
        AttributesDFList.append(df)
    
    # Combine all the dataframes into one
    AttributesDF = pd.concat(AttributesDFList, ignore_index=True)
    
    # Create a .csv (label file with z to insure it stays at the end of the directory)
    AttributesDF.to_csv(f'./data/player-attributes-lists/zMaster-Player-Attributes-List.csv', index = False, header = False)
    
    # Read the csv back for testing purposes
    AttributesCSV = pd.read_csv(f'./data/player-attributes-lists/zMaster-Player-Attributes-List.csv', header = None)
    AttributesCSV.columns = ['Name', 'Team', 'POS', 'HT', 'WT', 'DOB', 'AGE', 'EXP', 'Key']
    print(AttributesCSV)
    
    return 0

In [5]:
# Method to get the revelent attributes for every player currently signed on every NFL team
# Reads in the team list csv, and calls the GetTeamSalaryCapHits method for each team
def GetAllPlayerAttributes():
    # Read in the full team list csv
    FullTeamListCSV = pd.read_csv('./data/team-attribute-lists/Teams-List.csv', header = None)
    FullTeamListCSV.columns = ['TeamName', 'Abbreviation', 'Conference', 'Division']
    
    # Create a list of all the team names and abbreviations as a list of lists
    TeamList = []
    # Take the first two columns of each row, convert to a list, append to TeamList
    FullTeamListCSV.apply(lambda row: TeamList.append(row[:2].tolist()), axis = 1)
    
    # Get the salary cap hits for each team
    for team in TeamList:
        # format the team name to be used in the URL
        team[0] = team[0].replace(" ", "-").lower()
        GetPlayterAttributes(team)
    

GetAllPlayerAttributes()
CombineAttributesLists()

Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with status code 200
Request succeeded with st

0